In [10]:
import cell_movie_maker as cmm
import chaste_simulation_database_connector as csdc
import matplotlib.pyplot as plt
import numpy as np
import pathlib


from IPython.display import display
import IPython.display


cmm.Config.simulations_folder = pathlib.Path("chaste_output")
cmm.Config.output_folder = pathlib.Path("analysis_output")

experiment = cmm.Experiment(pathlib.Path("chaste_output", "TCellABM"))
simulation = experiment.simulations[0]
sample_timepoint = simulation.timepoints[50]


# Creating and Writing to the Database
For reading from the database see next example  

This requires chaste-simulation-database-connector module

Setting the database path in cmm.Config will create the database if it does not exist

In [11]:
cmm.Config.set_simulation_database(pathlib.Path("analysis_output", "TCellABM.db"))

# Writing Parameters

In [12]:
ingest = cmm.csdc.ParametersIngest(cmm.Config.simulation_database)
ingest.ingest_experiment(experiment)
cmm.Config.simulation_database.commit()

100%|██████████| 1/1 [00:00<00:00, 1051.73it/s]


# Writing Preprocessing
Framework for saving preprocessing to the database is separate from analysis (legacy reasons)  
However it is stored in the same place as other analysis, and reading the data back can be done identical to other analysis methods.  

In [13]:
ingest = cmm.csdc.InfoIngester(cmm.Config.simulation_database)
ingest.ingest_experiment(experiment)
cmm.Config.simulation_database.commit()

100%|██████████| 1/1 [00:00<00:00, 295.27it/s]


# Writing Analysis
Depending on whether each analysis_result should be a table for each timepoint (e.g. PCF), or a table with rows as timesteps and columns as analysis results (e.g. roundness or cellcounts)  
We use either `TimepointAnalysisIngest` or `SimulationAnalysisIngest`.  

Many of these classes will make use of multiprocessing to parallelize analysis (parameter nproc).  
By default if analysis has already been performed it will not be performed again (parameter skip_existing).  
batch_size parameter is how many simulations to process at a time before saving to the database, this allows analysis ingester to restore midway if there is a crash halfway through processing a large parameter sweep.  

## `SimulationAnalysisIngest`

### Roundness

In [14]:
analyser = cmm.analysers.MultiTimepointAnalyser(
    cmm.analysers.RoundnessAnalyser(.5), slice(None,None,-1))
ingest = cmm.csdc.SimulationAnalysisIngest(cmm.Config.simulation_database, skip_existing=False)
ingest.ingest_experiment(experiment, analyser)
cmm.Config.simulation_database.commit()

100%|██████████| 1/1 [00:37<00:00, 37.13s/it]


### Tumour Size Analysis

In [17]:
analyser = cmm.analysers.MultiTimepointAnalyser(
    cmm.analysers.TumourRegionSizesAnalyser(.5), slice(None, None, -1))
ingest = cmm.csdc.SimulationAnalysisIngest(cmm.Config.simulation_database, skip_existing=False)
ingest.ingest_experiment(experiment, analyser)
cmm.Config.simulation_database.commit()

100%|██████████| 1/1 [00:43<00:00, 43.34s/it]


### Tumour Region T-Cell Counts

In [15]:
analyser = cmm.analysers.MultiTimepointAnalyser(
    cmm.analysers.TumourRegionTCellCountAnalyser(.5), slice(None, None, -1))
ingest = cmm.csdc.SimulationAnalysisIngest(cmm.Config.simulation_database, skip_existing=False)
ingest.ingest_experiment(experiment, analyser)
cmm.Config.simulation_database.commit()

100%|██████████| 1/1 [00:47<00:00, 47.76s/it]


## `TimepointAnalysisIngest`
### PCF Analysis
Analyse every 5th timepoint

In [16]:
ingest = cmm.csdc.TimepointAnalysisIngest(cmm.Config.simulation_database, timestep_slice=slice(None,None,-5), skip_existing=True)

print("Tumour-Vessel")
ingest.ingest_experiment(
    experiment, 
    cmm.analysers.TumourVesselPCFAnalyser(r_max=50, dr=1, step=1))

print("Tumour-Tumour")
ingest.ingest_experiment(
    experiment, 
    cmm.analysers.TumourTumourPCFAnalyser(r_max=50, dr=1, step=1))

print("TCell-TCell")
ingest.ingest_experiment(
    experiment, 
    cmm.analysers.TCellTCellPCFAnalyser(r_max=50, dr=1, step=1))

print("Vessel-TCell")
ingest.ingest_experiment(
    experiment, 
    cmm.analysers.VesselTCellPCFAnalyser(r_max=50, dr=1, step=1))

print("Tumour-TCell")
ingest.ingest_experiment(
    experiment, 
    cmm.analysers.TumourTCellPCFAnalyser(r_max=50, dr=1, step=1))



Tumour-Vessel


Performing analysis: 100%|██████████| 21/21 [00:30<00:00,  1.43s/it]


Tumour-Tumour


Performing analysis: 100%|██████████| 21/21 [00:33<00:00,  1.59s/it]


TCell-TCell


Performing analysis: 100%|██████████| 21/21 [00:40<00:00,  1.92s/it]


Vessel-TCell


Performing analysis: 100%|██████████| 21/21 [00:04<00:00,  4.39it/s]


Tumour-TCell


Performing analysis: 100%|██████████| 21/21 [00:32<00:00,  1.55s/it]
